In [1]:
import numpy as np
import matplotlib as plt

from tda.sim.sim_engine import Simulation
from tda.sim.sim_objects.lti_object import LTIObject
from tda.sim.sensors.oracle import oracle

ImportError: cannot import name 'SimObject' from partially initialized module 'tda.sim.sim_objects.sim_object' (most likely due to a circular import) (/home/erik/devel/TDA/.venv/lib64/python3.11/site-packages/tda/sim/sim_objects/sim_object.py)